# Setting Up `neurodesign` and `deconvolve`

To work with the `neurodesign` and `deconvolve` packages for experiment creation, we recommend setting up a Python virtual environment. This ensures that your project dependencies remain isolated and reproducible.

---

**I. Clone and open the repository folder in Visual Studio Code.**

---

**II. Create a virtual environment**

You can do this either through the VS Code interface or directly in the terminal.

**A. Using VS Code GUI:**
1. Open the Command Palette:  
   `View > Command Palette`
2. Search for and select:  
   `Python: Select Interpreter`
3. Click:  
   `+ Create Environment`
4. Choose the environment type:  
   `venv`
5. Select a Python interpreter (version `>= 3.9`, e.g., `Python 3.13.1`)

**B. Using the Terminal:**

From the root of the project folder, run:
```bash
python -m venv venv
```

---

**III. Activate the virtual environment**

- On macOS/Linux:
  ```bash
  source venv/bin/activate
  ```
- On Windows:
  ```bash
  venv\Scripts\Activate.ps1
  ```

---

**IV. Install required packages**

With the virtual environment activated, install dependencies:
```bash
pip install git+https://github.com/neuropower/neurodesign.git
pip install -r requirements.txt
```

To deactivate the environment when you're done:
```bash
deactivate
```

---

**V. (Optional) Use the virtual environment in Jupyter Notebooks**

If you're working in `.ipynb` notebooks, register the virtual environment as a Jupyter kernel:
```bash
python -m ipykernel install --user --name=venv --display-name "Python (neurodesign venv)"
```

Reload VSC. Then, in Jupyter, select the `"Python (neurodesign venv)"` kernel when working on notebooks.

---

**Note:** At this stage, we are primarily using the `neurodesign` package.

---

## Design Setup and Automatic Optimization


Commented code taken from the tutorial folder for Neurodesign GitHub [repository](https://github.com/neuropower/neurodesign/tree/master/examples)

We automatically generate and evaluate optimal fMRI experimental designs using a genetic algorithm, and then export a detailed PDF report summarizing the results.

---

### Step 1: Initial imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import neurodesign

In [2]:
import os
# Since Neurodesign uses multi-threading internally, limit the number of threads
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

### Step 2: Define the Experimental Setup
- **Experiment Object (`exp`)**: The experimental parameters are defined here:
  - **TR (Repetition Time)** is set to 2 seconds.
  - **n_trials**: The experiment consists of 100 trials.
  - **P**: The probabilities for each of the three conditions are set to 0.33 (equal probability).
  - **C**: The contrast matrix specifies the contrasts between conditions (e.g., condition 1 vs. condition 2).
  - **n_stimuli**: There are 3 distinct stimuli presented in the experiment.
  - **rho**: The autocorrelation between trials is set to 0.3, which models the correlation between consecutive trials.
  - **stim_duration**: Each stimulus is presented for 1 second.
  - **t_pre**: There is no time before the stimulus onset (t_pre=0).
  - **t_post**: A post-stimulus duration of 2 seconds is included.
  - **ITI Model**: The inter-trial interval (ITI) is modeled using an exponential distribution.
  - **ITImin, ITImean, ITImax**: The inter-trial interval is between 1 and 4 seconds, with a mean of 2 seconds.

In [10]:
from pathlib import Path

from neurodesign import Experiment, Optimisation, report

# Define the output directory where results will be saved
output_dir = Path("./output/tuto_optimizing_designs")
output_dir.mkdir(parents=True, exist_ok=True)

# -------------------------------
# DEFINE EXPERIMENT PARAMETERS
# -------------------------------

# Create an Experiment object specifying the experimental setup
exp = Experiment(
    TR=2,  # Repetition time (in seconds)
    n_trials=100,  # Number of trials in the experiment
    P=[0.33, 0.33, 0.33],  # Probabilities for three conditions (equally likely)
    C=[[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, -1, 0], [0, 1, -1]],  # Contrast matrix
    n_stimuli=3,  # Number of distinct stimuli
    rho=0.3,  # Autocorrelation between trials (correlation structure)
    resolution=0.1,  # Temporal resolution of the design
    stim_duration=1,  # Duration of each stimulus presentation (in seconds)
    t_pre=0,  # Time before stimulus onset
    t_post=2,  # Time after stimulus offset
    restnum=0,  # Number of rest periods
    restdur=0,  # Duration of rest periods
    ITImodel="exponential",  # Model for inter-trial interval (exponential distribution)
    ITImin=1,  # Minimum inter-trial interval (in seconds)
    ITImean=2,  # Mean inter-trial interval (in seconds)
    ITImax=4,  # Maximum inter-trial interval (in seconds)
)

c:\Users\vguigon\Desktop\Research_directory\Lab_SLD\sld_simulating_fmri\venv\Lib\site-packages\neurodesign\classes.py:484: UserWarning: the resolution is adjusted to be a multiple of the TR. New resolution: 0.1
  warnings.warn(
c:\Users\vguigon\Desktop\Research_directory\Lab_SLD\sld_simulating_fmri\venv\Lib\site-packages\neurodesign\classes.py:643: RuntimeWarning: divide by zero encountered in log
  res = (h - 1) * np.log(s) + h * np.log(l) - l * s - np.log(gamma(h))


### Step 3: Setup the Optimization Parameters
- **Optimisation Object (`population`)**: This object defines the optimization parameters:
  - **weights**: The relative importance of different factors during the optimization (Fd, Ff, confounding, and frequency balance). Fd and Ff are weighted more heavily (0.5) compared to the others (0.25).
  - **preruncycles**: 10 warm-up cycles are run before the main optimization.
  - **cycles**: The main optimization is run for 10 cycles.
  - **seed**: A random seed (1) is used for reproducibility of results.
  - **outdes**: The top 5 designs will be output after the optimization.
  - **folder**: The optimization results will be saved in the `output` folder.

In [5]:
# -------------------------------
# SETUP OPTIMIZATION PARAMETERS
# -------------------------------

# Create an Optimisation object that will be used to optimize the experimental design
population = Optimisation(
    experiment=exp,  # Link to the experiment defined above
    weights=[0, 0.5, 0.25, 0.25],  # Weights for Fd, Ff, confounding, and frequency (importance of each metric)
    preruncycles=10,  # Number of "warm-up" iterations before starting the optimization
    cycles=10,  # Number of optimization cycles (main iterations)
    seed=1,  # Random seed for reproducibility
    outdes=5,  # Number of output designs to return
    folder=output_dir,  # Folder where the optimization results will be saved
)

### Step 4: Run the Optimization (Natural Selection Process)
- The **optimisation** process is executed via the `optimise()` method, which runs a genetic algorithm to find the best experimental design.
- **Download**: The best designs, along with their related data, are downloaded.
- **Evaluate**: The optimized designs are evaluated based on the defined metrics (Fd, Ff, etc.).

In [6]:
# -------------------------------
# RUN NATURAL SELECTION (GENETIC ALGORITHM)
# -------------------------------

# Execute the optimization process
population.optimise()

# Download the best designs and their related data
population.download()

# Evaluate the optimization results (e.g., check efficiency metrics)
population.evaluate()

c:\Users\vguigon\Desktop\Research_directory\Lab_SLD\sld_simulating_fmri\venv\Lib\site-packages\rich\live.py:231: 
UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

### Step 5: Step-by-Step Simulation of Evolution
- **add_new_designs()**: New designs are added to the population after optimization.
- **to_next_generation()**: The population moves to the next generation, evolving the designs over two iterations with different random seeds.

In [7]:
# -------------------------------
# STEP BY STEP SIMULATION OF EVOLUTION
# -------------------------------

# Add new designs to the population (based on optimization process)
population.add_new_designs()

# Move to the next generation of designs using a specific random seed for reproducibility
population.to_next_generation(seed=1)
population.to_next_generation(seed=1001)

### Step 6: Export Final Report
- The final **PDF report** is generated using the `report.make_report()` function, summarizing the optimization results and the best designs found. The report is saved in the `output` folder as `test.pdf`.

In [9]:
# -------------------------------
# EXPORT FINAL REPORT
# -------------------------------

# Generate a PDF report summarizing the results of the optimization
report.make_report(population, output_dir / "tuto_optimizing_designs_report.pdf")